# Tarea 4
## Por Juan Pablo Morales Gomez

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
import numpy as np
import subprocess
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [2]:
img_width, img_height = 196, 196
train_data_dir = './data/train'
validation_data_dir = './data/validation'
nb_train_samples =4750
nb_validation_samples = 1195
epochs = 10
batch_size = 16

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:

model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
 
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 4750 images belonging to 12 classes.
Found 1195 images belonging to 12 classes.


C:\Users\jmora\AppData\Local\Temp\ipykernel_35596\3119093969.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
296/296 [==============================] - 191s 640ms/step - loss: -256294400.0000 - accuracy: 0.0820 - val_loss: -1298627712.0000 - val_accuracy: 0.0845
Epoch 2/10
296/296 [==============================] - 181s 611ms/step - loss: -8547578880.0000 - accuracy: 0.0820 - val_loss: -23753689088.0000 - val_accuracy: 0.0819
Epoch 3/10
296/296 [==============================] - 178s 602ms/step - loss: -69431427072.0000 - accuracy: 0.0822 - val_loss: -143524806656.0000 - val_accuracy: 0.0811
Epoch 4/10
296/296 [==============================] - 183s 618ms/step - loss: -301852950528.0000 - accuracy: 0.0817 - val_loss: -527739748352.0000 - val_accuracy: 0.0828
Epoch 5/10
296/296 [==============================] - 191s 645ms/step - loss: -929587200000.0000 - accuracy: 0.0817 - val_loss: -1474298118144.0000 - val_accuracy: 0.0819
Epoch 6/10
296/296 [==============================] - 190s 641ms/step - loss: -2353806180352.0000 - accuracy: 0.0822 - val_loss: -3440121217024.0000 - val_acc

In [7]:
model.save_weights('model_saved.h5')

In [8]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import csv
import tensorflow as tf



In [10]:
directory = './data/test'
model_path = './model_saved.h5'
dataset = []

for filename in os.listdir(directory):
    if filename.endswith('.png'):
        
        image_path = os.path.join(directory, filename)
        image = Image.open(image_path)
        image = image.resize((196, 196))  

        image = np.array(image)
        image = image / 255.0  
        image = np.expand_dims(image, axis=0)

       
        prediction = model.predict(image)
        predicted_label = np.argmax(prediction)

       
        dataset.append((image_path, predicted_label))


csv_filename = 'dataset.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Label'])
    writer.writerows(dataset)



1/1 [==============================] - 0s 35ms/step
El conjunto de datos se ha guardado correctamente en dataset.csv


In [12]:
label_mapping = {0: 'Black-grass', 1: 'Charlock',1: 'Charlock',1: 'Charlock',1: 'Charlock',1: 'Charlock',1: 'Charlock',1: 'Charlock',1: 'Charlock',}

transformed_data = []

with open(csv_filename, 'r') as file:
    reader = csv.reader(file)
    header = next(reader)  
    transformed_data.append(['file', 'species']) 

    for row in reader:
        image_path = row[0]
        label = int(row[1])

        image_filename = os.path.basename(image_path)

        transformed_label = label_mapping.get(label, '')

        transformed_data.append([image_filename, transformed_label])

new_csv_filename = 'transformed_dataset.csv'
with open(new_csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(transformed_data)